In [1]:
import sys
# use line-buffering for both stdout and stderr
# sys.stdout = open(sys.stdout.fileno(), mode='w', buffering=1)
# sys.stderr = open(sys.stderr.fileno(), mode='w', buffering=1)

import hydra
from omegaconf import OmegaConf
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import pathlib
from diffusion_policy.workspace.base_workspace import BaseWorkspace
 
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
# from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.dataset.base_dataset import BaseLowdimDataset
from diffusion_policy.env_runner.base_lowdim_runner import BaseLowdimRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to

from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.policy.robomimic_image_policy import RobomimicImagePolicy
from diffusion_policy.dataset.base_dataset import BaseImageDataset
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner

# from diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace import TrainDiffusionUnetHybridWorkspace
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.diffusion_unet_hybrid_image_policy import DiffusionUnetHybridImagePolicy
from diffusion_policy.dataset.base_dataset import BaseImageDataset
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to
from diffusion_policy.model.diffusion.ema_model import EMAModel
from diffusion_policy.model.common.lr_scheduler import get_scheduler

from diffusion_policy.dataset.robomimic_replay_image_dataset import RobomimicReplayImageDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Sampler 

import h5py

# import mimicgen
# import mimicgen.utils.file_utils as MG_FileUtils
# import mimicgen.utils.robomimic_utils as RobomimicUtils
# from mimicgen.utils.misc_utils import add_red_border_to_frame
# from mimicgen.configs import MG_TaskSpec

OmegaConf.register_new_resolver("eval", eval, replace=True)

In [2]:
config_path='.'
config_name = "image_franka_coffee_vr.yaml" 

In [3]:
with initialize(version_base=None, config_path=config_path):
    cfg_org = compose(
        config_name=config_name,
        overrides=[
            "hydra.run.dir=data/outputs/${now:%Y.%m.%d}/${now:%H.%M.%S}_${name}_${task_name}",
            "training.seed=42",
            "training.device=cuda:0"
        ],
    )
    print(cfg_org)
    
OmegaConf.resolve(cfg_org)

print('resume: ', cfg_org.training.resume)

{'_target_': 'diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace.TrainDiffusionUnetHybridWorkspace', 'checkpoint': {'save_last_ckpt': True, 'save_last_snapshot': False, 'topk': {'format_str': 'epoch={epoch:04d}-test_mean_score={test_mean_score:.3f}.ckpt', 'k': 5, 'mode': 'max', 'monitor_key': 'test_mean_score'}}, 'dataloader': {'batch_size': 64, 'num_workers': 8, 'persistent_workers': False, 'pin_memory': True, 'shuffle': True}, 'dataset_obs_steps': 2, 'ema': {'_target_': 'diffusion_policy.model.diffusion.ema_model.EMAModel', 'inv_gamma': 1.0, 'max_value': 0.9999, 'min_value': 0.0, 'power': 0.75, 'update_after_step': 0}, 'exp_name': 'default', 'horizon': 16, 'keypoint_visible_rate': 1.0, 'logging': {'group': None, 'id': None, 'mode': 'online', 'name': '2022.12.29-22.31.41_train_diffusion_unet_hybrid_square_image', 'project': 'diffusion_policy_debug', 'resume': True, 'tags': ['train_diffusion_unet_hybrid', 'square_image', 'default']}, 'multi_run': {'run_dir': 'data/outputs

In [ ]:
# last_checkpoint_dir = "/home/carl_lab/diffusion_policy/data/outputs/2024.12.13/03.05.17_train_diffusion_unet_hybrid_square_image/"
last_checkpoint_dir = None 

In [4]:
class TrainDiffusionUnetHybridWorkspace(BaseWorkspace):
    include_keys = ['global_step', 'epoch']

    def __init__(self, cfg: OmegaConf, output_dir=None):
        super().__init__(cfg, output_dir=output_dir)

        # set seed
        seed = cfg.training.seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # configure model
        self.model: DiffusionUnetHybridImagePolicy = hydra.utils.instantiate(cfg.policy)

        self.ema_model: DiffusionUnetHybridImagePolicy = None
        if cfg.training.use_ema:
            self.ema_model = copy.deepcopy(self.model)

        # configure training state
        self.optimizer = hydra.utils.instantiate(
            cfg.optimizer, params=self.model.parameters())

        # configure training state
        self.global_step = 0
        self.epoch = 0

In [5]:
import datetime

### recreating workspace

In [6]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"/home/carl_lab/diffusion_policy/data/outputs/devin-custom{timestamp}"
os.mkdir(output_dir)
workspace = TrainDiffusionUnetHybridWorkspace(cfg_org, output_dir=output_dir)

self = workspace


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['ee_states', 'gripper_states', 'joint_states']
using obs modality: rgb with keys: ['eye_in_hand_rgb', 'agentview_rgb']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/home/carl_lab/miniconda3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/carl_lab/miniconda3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Diffusion params: 2.564722e+08
Vision params: 2.239418e+07


In [7]:
cfg = copy.deepcopy(self.cfg)

# resume training
# if cfg.training.resume:
#     lastest_ckpt_path = self.get_checkpoint_path()
#     if lastest_ckpt_path.is_file():
#         print(f"Resuming from checkpoint {lastest_ckpt_path}")
#         self.load_checkpoint(path=lastest_ckpt_path)

In [8]:
new_config = OmegaConf.to_container(cfg.task.dataset, resolve=True )
del new_config['_target_']
# new_config['shape_meta']['obs']['demo_no']={'shape':[] }
# new_config['shape_meta']['obs']['index_in_demo']={'shape':[] }


dataset = RobomimicReplayImageDataset(**new_config)
len(dataset)

Acquiring lock on cache.
Cache does not exist. Creating!


Loading image data: 100%|██████████| 67810/67810 [01:54<00:00, 591.18it/s]


Saving cache to disk.


32322

In [9]:
cfg_dataloader = {key:value for key,value in cfg.dataloader.items()} 

In [10]:
train_dataloader = DataLoader(dataset, **cfg_dataloader)
normalizer = dataset.get_normalizer()

# configure validation dataset
val_dataset = dataset.get_validation_dataset()
val_dataloader = DataLoader(val_dataset, **cfg.val_dataloader)

In [11]:
batch =  next(iter(train_dataloader))
batch.keys()

dict_keys(['obs', 'action'])

In [12]:
batch['action'].shape

torch.Size([64, 16, 10])

In [13]:
batch['obs']['agentview_rgb'].shape

torch.Size([64, 2, 3, 240, 320])

In [14]:
self.model.set_normalizer(normalizer)
if cfg.training.use_ema:
    self.ema_model.set_normalizer(normalizer)

# configure lr scheduler
lr_scheduler = get_scheduler(
    cfg.training.lr_scheduler,
    optimizer=self.optimizer,
    num_warmup_steps=cfg.training.lr_warmup_steps,
    num_training_steps=(
        len(train_dataloader) * cfg.training.num_epochs) \
            // cfg.training.gradient_accumulate_every,
    # pytorch assumes stepping LRScheduler every epoch
    # however huggingface diffusers steps it every batch
    last_epoch=self.global_step-1
)

# configure ema
ema: EMAModel = None
if cfg.training.use_ema:
    ema = hydra.utils.instantiate(
        cfg.ema,
        model=self.ema_model)

In [15]:
topk_manager = TopKCheckpointManager(
    save_dir=os.path.join(self.output_dir, 'checkpoints'),
    **cfg.checkpoint.topk
)

# device transfer
device = torch.device(cfg.training.device)
self.model.to(device)
if self.ema_model is not None:
    self.ema_model.to(device)
optimizer_to(self.optimizer, device)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: [0.95, 0.999]
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.0001
    lr: 0.0
    maximize: False
    weight_decay: 1e-06
)

In [16]:
train_sampling_batch = None
log_path = os.path.join(self.output_dir, 'logs.json.txt')
with JsonLogger(log_path) as json_logger:
    for local_epoch_idx in range(cfg.training.num_epochs):
        step_log = dict()
        # ========= train for this epoch ==========
        train_losses = list()
        with tqdm.tqdm(train_dataloader, desc=f"Training epoch {self.epoch}", 
                leave=False, mininterval=cfg.training.tqdm_interval_sec) as tepoch:
            for batch_idx, batch in enumerate(tepoch):
                # device transfer
                batch = dict_apply(batch, lambda x: x.to(device, non_blocking=True))
                if train_sampling_batch is None:
                    train_sampling_batch = batch

                # compute loss
                raw_loss = self.model.compute_loss(batch)
                loss = raw_loss / cfg.training.gradient_accumulate_every
                loss.backward()

                # step optimizer
                if self.global_step % cfg.training.gradient_accumulate_every == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    lr_scheduler.step()
                
                # update ema
                if cfg.training.use_ema:
                    ema.step(self.model)

                # logging
                raw_loss_cpu = raw_loss.item()
                tepoch.set_postfix(loss=raw_loss_cpu, refresh=False)
                train_losses.append(raw_loss_cpu)
                step_log = {
                    'train_loss': raw_loss_cpu,
                    'global_step': self.global_step,
                    'epoch': self.epoch,
                    'lr': lr_scheduler.get_last_lr()[0]
                }

                is_last_batch = (batch_idx == (len(train_dataloader)-1))
                if not is_last_batch:
                    # log of last step is combined with validation and rollout
                     
                    json_logger.log(step_log)
                    self.global_step += 1

                if (cfg.training.max_train_steps is not None) \
                    and batch_idx >= (cfg.training.max_train_steps-1):
                    break

        # at the end of each epoch
        # replace train_loss with epoch average
        train_loss = np.mean(train_losses)
        step_log['train_loss'] = train_loss

        # ========= eval for this epoch ==========
        policy = self.model
        if cfg.training.use_ema:
            policy = self.ema_model
        policy.eval()

 
        # run validation
        if (self.epoch % cfg.training.val_every) == 0:
            with torch.no_grad():
                val_losses = list()
                with tqdm.tqdm(val_dataloader, desc=f"Validation epoch {self.epoch}", 
                        leave=False, mininterval=cfg.training.tqdm_interval_sec) as tepoch:
                    for batch_idx, batch in enumerate(tepoch):
                        batch = dict_apply(batch, lambda x: x.to(device, non_blocking=True))
                        loss = self.model.compute_loss(batch)
                        val_losses.append(loss)
                        if (cfg.training.max_val_steps is not None) \
                            and batch_idx >= (cfg.training.max_val_steps-1):
                            break
                if len(val_losses) > 0:
                    val_loss = torch.mean(torch.tensor(val_losses)).item()
                    # log epoch average validation loss
                    step_log['val_loss'] = val_loss

        # run diffusion sampling on a training batch
        if (self.epoch % cfg.training.sample_every) == 0:
            with torch.no_grad():
                # sample trajectory from training set, and evaluate difference
                batch = dict_apply(train_sampling_batch, lambda x: x.to(device, non_blocking=True))
                obs_dict = batch['obs']
                gt_action = batch['action']
                
                result = policy.predict_action(obs_dict)
                pred_action = result['action_pred']
                mse = torch.nn.functional.mse_loss(pred_action, gt_action)
                step_log['train_action_mse_error'] = mse.item()
                del batch
                del obs_dict
                del gt_action
                del result
                del pred_action
                del mse
        
        # checkpoint
        if (self.epoch % cfg.training.checkpoint_every) == 0:
            # checkpointing
            if cfg.checkpoint.save_last_ckpt:
                self.save_checkpoint()
            if cfg.checkpoint.save_last_snapshot:
                self.save_snapshot()

            # sanitize metric names
            metric_dict = dict()
            for key, value in step_log.items():
                new_key = key.replace('/', '_')
                metric_dict[new_key] = value
            
            
        # ========= eval end for this epoch ==========
        policy.train()

        # end of epoch
        # log of last step is combined with validation and rollout
         
        json_logger.log(step_log)
        self.global_step += 1
        self.epoch += 1

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│    16 │   │   │   │   # compute loss                                                             │
│    17 │   │   │   │   raw_loss = self.model.compute_loss(batch)                                  │
│    18 │   │   │   │   loss = raw_loss / cfg.training.gradient_accumulate_every                   │
│ ❱  19 │   │   │   │   loss.backward()                                                            │
│    20 │   │   │   │                                                                              │
│    21 │   │   │   │   # step optimizer                                                           │
│    22 │   │   │   │   if self.global_step % cfg.training.gradient_accumulate_every == 0:         │
│                                                                                                  │
│ /home/carl_lab/miniconda3/envs/robodiff/lib/python3.9/site-packages/torch/_tensor.py:396 in      │
│ backward                                                                                         │
│                                                                                                  │
│    393 │   │   │   │   retain_graph=retain_graph,                                                │
│    394 │   │   │   │   create_graph=create_graph,                                                │
│    395 │   │   │   │   inputs=inputs)                                                            │
│ ❱  396 │   │   torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=input  │
│    397 │                                                                                         │
│    398 │   def register_hook(self, hook):                                                        │
│    399 │   │   r"""Registers a backward hook.                                                    │
│                                                                                                  │
│ /home/carl_lab/miniconda3/envs/robodiff/lib/python3.9/site-packages/torch/autograd/__init__.py:1 │
│ 73 in backward                                                                                   │
│                                                                                                  │
│   170 │   # The reason we repeat same the comment below is that                                  │
│   171 │   # some Python versions print out the first line of a multi-line function               │
│   172 │   # calls in the traceback and some print out the last line                              │
│ ❱ 173 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   174 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   175 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   176                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [17]:
self.save_checkpoint(tag="after_train")

'/home/carl_lab/diffusion_policy/data/outputs/devin-custom20250721_123045/checkpoints/after_train.ckpt'

In [19]:
cfg.training.checkpoint_every, cfg.checkpoint.save_last_ckpt

(50, True)